# Validate Motion State is Working

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import fetch_data
import numpy as np
import correct_drift
import datetime

import importlib
importlib.reload(fetch_data)

<module 'fetch_data' from '/home/tyler/Documents/Matt3r/IMU_validation/fetch_data.py'>

Select Data

In [2]:
# Hamid data (must be after July 7th)
start_date_str = '2023-07-07'
end_date_str = '2023-07-08'
organization_id = 'hamid'
can_k3y_id = '17700cf8'
imu_k3y_id = '17700cf8'

In [ ]:
event_dict = fetch_data.get_events(can_k3y_id, organization_id, start_date_str, end_date_str)
can_df = fetch_data.get_can_data(can_k3y_id, organization_id, start_date_str, end_date_str)
imu_df = fetch_data.get_imu_data(imu_k3y_id, organization_id, start_date_str, end_date_str)
imu_df['norm_acc'] = np.sqrt(imu_df['lr_acc(m/s^2)']**2 + imu_df['bf_acc(m/s^2)']**2 + imu_df['vert_acc(m/s^2)']**2)

Use the speed data to create stationary and driving states

In [ ]:
# get the driving state data based on the CAN Server speed data

stationary_speed = 0.5
drive_duration = 0.5

speed_df = can_df[can_df['speed'].notna()].copy()
speed_df.reset_index(drop=True, inplace=True)
speed_df['driving'] = abs(speed_df['speed']) > stationary_speed
dr_start_times = speed_df[speed_df['driving'].astype(int).diff() == 1]['timestamp'].to_list()
dr_end_times = speed_df[speed_df['driving'].astype(int).diff() == -1]['timestamp'].to_list()

# get the driving states
dr_df_states = []
for i in range(min(len(dr_start_times),len(dr_end_times))):
    if dr_end_times[i] - dr_start_times[i] > drive_duration:
        dr_df_states.append(imu_df[(imu_df['correct_timestamp'] >= dr_start_times[i]) 
                        & (imu_df['correct_timestamp'] <= dr_end_times[i])])
dr_df = pd.concat(dr_df_states, ignore_index=True)